In [1]:
import import_ipynb
import numpy as np
import pdb
import math
import Datagenerators as D
import copy

importing Jupyter notebook from Datagenerators.ipynb
importing Jupyter notebook from DataAugment.ipynb


Using TensorFlow backend.


importing Jupyter notebook from pretrained_model.ipynb


In [17]:
def apply_rpn(cls_layer, regr_layer, max_boxes=300,overlap_thresh=0.9):
    regr_layer/=4
    anc_sizes=[128, 256, 512]
    anc_ratios=[[1, 1], [1, 2], [2, 1]]
    curr_layer=0
    (rows, cols) = cls_layer.shape[1:3]
    B=np.zeros((4,cls_layer.shape[1],cls_layer.shape[2],cls_layer.shape[3]))
    for a_s in anc_sizes:
        for a_r in anc_ratios:
            anc_x=(a_s*a_r[0])/16
            anc_y=(a_s*a_r[1])/16
            regr=np.transpose(regr_layer[0, :, :, 4 * curr_layer:4 * curr_layer + 4],(2,0,1))
            X, Y = np.meshgrid(np.arange(cols),np. arange(rows))
            B[0, :, :, curr_layer] = X - anc_x/2
            B[1, :, :, curr_layer] = Y - anc_x/2
            B[2, :, :, curr_layer] = anc_x
            B[3, :, :, curr_layer] = anc_y
            d=B[:, :, :, curr_layer]
            try:
                x=d[0,:,:]
                y=d[1,:,:]
                w=d[2,:,:]
                h=d[3,:,:]
                tx=regr[0,:,:]
                ty=regr[1,:,:]
                tw=regr[2,:,:]
                th=regr[3,:,:]
                cx=x+w/2
                cy=y+h/2
                cx1=tx*w+cx
                cy1=ty*h+cy
                w1 = np.exp(tw.astype(np.float64)) * w
                h1 = np.exp(tw.astype(np.float64)) * h
                x1 = cx1 - w1/2.
                y1 = cy1 - h1/2.
                x1 = np.round(x1)
                y1 = np.round(y1)
                w1 = np.round(w1)
                h1 = np.round(h1)
                B[:, :, :, curr_layer]=np.stack([x1, y1, w1, h1])
            except Exception as e:
                print(e)
            B[2, :, :, curr_layer] = np.maximum(1, B[2, :, :, curr_layer])
            B[3, :, :, curr_layer] = np.maximum(1, B[3, :, :, curr_layer])
            B[2, :, :, curr_layer] += B[0, :, :, curr_layer]
            B[3, :, :, curr_layer] += B[1, :, :, curr_layer]
            B[0, :, :, curr_layer] = np.maximum(0, B[0, :, :, curr_layer])
            B[1, :, :, curr_layer] = np.maximum(0, B[1, :, :, curr_layer])
            B[2, :, :, curr_layer] = np.minimum(cols-1, B[2, :, :, curr_layer])
            B[3, :, :, curr_layer] = np.minimum(rows-1, B[3, :, :, curr_layer])
            
            curr_layer+=1
            all_boxes = np.reshape(B.transpose((0, 3, 1,2)), (4, -1)).transpose((1, 0))
            all_probs = cls_layer.transpose((0, 3, 1, 2)).reshape((-1))
            x1 = all_boxes[:, 0]
            y1 = all_boxes[:, 1]
            x2 = all_boxes[:, 2]
            y2 = all_boxes[:, 3]
            
            index = np.where((x1 - x2 >= 0) | (y1 - y2 >= 0))
            all_boxes = np.delete(all_boxes, index, 0)
            all_probs = np.delete(all_probs, index, 0)
            R = nms(all_boxes, all_probs, overlap_thresh=0.9, max_boxes=300)[0]
            return R

In [18]:
def nms(boxes, probs, overlap_thresh=0.9, max_boxes=300):
    if len(boxes)==0:
        return []
    x1=boxes[:,0]
    y1=boxes[:,1]
    x2=boxes[:,2]
    y2=boxes[:,3]
    boxes=boxes.astype("float")
    fwd=[]
    area=abs((x2-x1)*(y2-y1))
    index=np.argsort(probs)
    while len(index)>0:
        last=len(index)-1
        fwd.append(index[len(index)-1])
        x_1 = np.maximum(x1[index[len(index)-1]], x1[index[:last]])
        y_1 = np.maximum(y1[index[len(index)-1]], y1[index[:last]])
        x_2 = np.minimum(x2[index[len(index)-1]], x2[index[:last]])
        y_2 = np.minimum(y2[index[len(index)-1]], y2[index[:last]])
        w_1 = np.maximum(0, x_2 - x_1)
        h_1 = np.maximum(0, y_2 - y_1)
        area_1=w_1*h_1
        area_u=area[index[len(index)-1]] + area[index[:last]] - area_1
        iou=area_1/(area_u+1e-6)
        index = np.delete(index, np.concatenate(([last],np.where(overlap_thresh>0.9)[0])))
        if len(fwd)>=max_boxes:
            break
    boxes=boxes[index].astype(int)
    probs=probs[index]
    return boxes,probs
    
        
    

In [19]:

import import_ipynb
import Datagenerators as D

In [20]:
def last_iou(R,img_data,class_mapping):
    bboxes=img_data['bboxes']
    (w,h)=(img_data['width'],img_data['height'])
    resized_w,resized_h=D.new_img_size(w, h, img_min_side=600)
    gtc=np.zeros((len(bboxes),4))
    for b_num,bbox in enumerate(bboxes):
        gtc[b_num, 0] = int(round(bbox['x1'] * (resized_w / float(w))/16))
        gtc[b_num, 1] = int(round(bbox['x2'] * (resized_w / float(w))/16))
        gtc[b_num, 2] = int(round(bbox['y1'] * (resized_h / float(h))/16))
        gtc[b_num, 3] = int(round(bbox['y2'] * (resized_h / float(h))/16))
    x_roi=[]
    y_cls_num=[]
    y_cls_regr_coords=[]
    y_cls_regr_label=[]
    iou=[]
    for i in range(R.shape[0]):
        x1,y1,x2,y2=R[i,:]
        best_iou=0.0
        best_bbox=-1
        for b_num in range(len(bboxes)):
            curr_iou=D.iou([gtc[b_num,0],gtc[b_num,2],gtc[b_num,1],gtc[b_num,3]],[x1,y1,x2,y2])
            if curr_iou>best_iou:
                best_iou=curr_iou
                best_bbox=b_num
        if best_iou<0.1:
            continue
        else:
            w=x2-x1
            h=y2-y1
            x_roi.append([x1,y1,w,h])
            iou.append(best_iou)
            if 0.1<=best_iou<0.5:
                cls_name='bg'
            elif 0.5<=best_iou:
                cls_name=bboxes[best_bbox]['class']
                cxg = (gtc[best_bbox, 0] + gtc[best_bbox, 1]) / 2.0
                cyg = (gtc[best_bbox, 2] + gtc[best_bbox, 3]) / 2.0
                cx=x1+w/2
                cy=y1+h/2
                tx = (cxg - cx) / float(w)
                ty =(cyg-cy)/float(w)
                tw = np.log((gtc[best_bbox, 1] - gtc[best_bbox, 0]) / float(w))
                th = np.log((gtc[best_bbox, 3] - gtc[best_bbox, 2]) / float(h))
        cls_num=class_mapping[cls_name]
        cls_label=len(class_mapping)*[0]
        cls_label[cls_num]=1
        y_cls_num.append(cls_label)          #due to roi pool individual
        coords = [0] * 4 * (len(class_mapping) - 1)            #one less lenght as 'bg' no cords and labesls
        labels = [0] * 4 * (len(class_mapping) - 1)
        if cls_name!='bg':
            label_pos = 4 * cls_num
            sx, sy, sw, sh= [8.0, 8.0, 4.0, 4.0]      #classifier_regr_std
            coords[label_pos:4+label_pos] = [sx*tx, sy*ty, sw*tw, sh*th]     
            labels[label_pos:4+label_pos] = [1, 1, 1, 1]
            y_cls_regr_coords.append(copy.deepcopy(coords))
            y_cls_regr_label.append(copy.deepcopy(labels))
        else:
            y_cls_regr_coords.append(copy.deepcopy(coords))
            y_cls_regr_label.append(copy.deepcopy(labels))
    if len(x_roi)==0:
        return None,None,None,None
    X=np.array(x_roi)
    Y1 = np.array(y_cls_num)
    Y2 = np.concatenate([np.array(y_cls_regr_label),np.array(y_cls_regr_coords)],axis=1)
    return np.expand_dims(X, axis=0), np.expand_dims(Y1, axis=0), np.expand_dims(Y2, axis=0), iou

In [6]:
#labels=[0] * 2 * (2)
#coords = [0] * 2 * (2)
#coords[0:2]=[2,3]
#labels[0:2]=[1,1]
#coords=[[2,3,0,0],[4,5,0,0]]
#labels=[[1,1,0,0],[0,0,1,1]]
#np.concatenate([np.array(coords),np.array(labels)],axis=1)

In [7]:
#import numpy as np
#np.meshgrid(np.arange(3),np.arange(2))